In [ ]:
from pathlib import Path

import matplotlib.pylab as plt
import pandas as pd

import utils

In [ ]:
pd.read_csv(utils.Configuration.RAW_DATA_PATH.joinpath("results.csv"))

In [ ]:
class Quantifyer:
    def __init__(
        self,
        quant_file,
        is_correspondence_file,
        qc_file=None,
        is_concentration_file=None,
    ):
        self.intact_quant_file = (
            pd.read_csv(quant_file)
            .rename(columns=lambda x: x.replace("-", "_").replace(" ", "_").lower())
            .assign(
                compound=lambda df: df.compound.replace("-", "_")
                .replace(" ", "_")
                .str.lower()
            )
            # .lower()
        )
        self.sample_volume = (
            pd.read_csv(quant_file).rename(
                columns=lambda x: x.replace("-", "_").replace(" ", "_").lower()
            )
            # .set_index("compound")
            # .T.query("sample_type.isin(['qc', 'sample', 'blank'])")
            # .volume.to_dict()
        )
        self.quant_file = pd.read_csv(quant_file)
        self.is_correspondence_file = pd.read_csv(is_correspondence_file)
        if qc_file is not None:
            self.qc_file = pd.read_csv(qc_file)
        else:
            self.qc_file = None

        if is_concentration_file is not None:
            self.is_concentration_file = pd.read_csv(is_concentration_file)
        else:
            self.is_concentration_file = None


quant = Quantifyer(
    quant_file=utils.Configuration.RAW_DATA_PATH.joinpath("results.csv"),
    is_correspondence_file=utils.Configuration.RAW_DATA_PATH.joinpath(
        "is_std_table_correspondence.csv"
    ),
)

In [ ]:
quant.intact_quant_file